<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:35%"><img src='https://dl.dropbox.com/s/qtzukmzqavebjd2/icon_smu.jpg' style="width: 300px; height: 90px; "></th>
    <th style="text-align:center;"><font size="4"> <br/>IS.215 - Analytics in Python Practical 1</font></th>
    </tr>
</table> 

This program builds a classifier for Pima Indians Diabetes dataset - https://www.kaggle.com/uciml/pima-indians-diabetes-database. It is a binary (2-class) classification problem. There are 768 observations with 8 input variables and 1 output/target variable. The variable names are as follows:

- Number of times pregnant.
- Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
- Diastolic blood pressure (mm Hg).
- Triceps skinfold thickness (mm).
- 2-Hour serum insulin (mu U/ml).
- Body mass index (weight in kg/(height in m)^2).
- Diabetes pedigree function.
- Age (years).
- Target variable (0 -'no' or 1-'yes').

In [ ]:
#Step 1: import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#Step 2: read in the data and analyse
df = pd.read_csv("diabetes.csv", names=['num_pregnant','glucose_conc','diastolic_bp','triceps_thick','serum_insulin','bmi','pedigree','age','class'])
print(df.shape)
print(df.describe())

In [ ]:
#Step 3: Split into input and target dataframes. axis=0, row, axis=1, column. 
input_df = ?
target = ?

print(input_df.shape,target.shape)

In [ ]:
#distribution of class
target.value_counts()

In [ ]:
#Step 4: Split feature and label sets to train and data sets - 70-30, random_state is desirable for reproducibility, stratify - same proportion as input data

X_train, X_test, y_train, y_test = train_test_split(input_df, target, test_size = 0.3, random_state = 10, stratify = target)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Question 2 - Normalize using MinMaxScaler to constrain values to between 0 and 1.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))

#?

In [ ]:
#Step 5: Create a logistic regression classifier, default c=1

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
#look at the value under the positive class - macro avg is insensitive to imbalanced data, micro result will be affected if there is imbalance data - infleunce by majority
print(classification_report(y_test, y_pred))

In [ ]:
#Question 4 - handling imbalance data - Rerunning above with resampled data - using oversampling
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 2)
X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train.ravel())

print(X_train_sm.shape, X_train.shape)

#?


In [ ]:
#Question 1,3 - analyse the features
#get the sorting indices in descending order
sorted_index = np.argsort(-logreg.coef_)

#get the feature_names
feature_names = input_df.columns

#get the names of the important features
#?